In [1]:
import pandas as pd
import numpy as np
import networkx as nx
pd.options.display.max_columns=1000

In [2]:
DATA_DIR = '../hike/'

In [3]:
train = pd.read_csv(DATA_DIR + 'train.csv', usecols=['node1_id','node2_id'], dtype={"is_chat": np.int8})

In [4]:
test = pd.read_csv(DATA_DIR + 'test.csv', usecols=['node1_id','node2_id'], dtype={"is_chat": np.int8})

In [5]:
train.shape, test.shape

((70661802, 2), (11776968, 2))

In [6]:
df = pd.concat([train, test], axis=0)

In [7]:
del train, test

In [8]:
%%time
graph = nx.from_pandas_edgelist(df=df, source='node1_id', target='node2_id', create_using=nx.DiGraph)

CPU times: user 13min 12s, sys: 44.4 s, total: 13min 56s
Wall time: 13min 54s


#### Calculating Degree

In [9]:
%%time
directed_degree = nx.algorithms.degree_centrality(G=graph)

CPU times: user 15.6 s, sys: 661 ms, total: 16.3 s
Wall time: 16.3 s


In [10]:
%%time
directed_degree_out = nx.algorithms.out_degree_centrality(G=graph)

CPU times: user 11.9 s, sys: 580 ms, total: 12.5 s
Wall time: 12.5 s


In [11]:
%%time
directed_degree_in = nx.algorithms.in_degree_centrality(G=graph)

CPU times: user 12.6 s, sys: 566 ms, total: 13.2 s
Wall time: 13.1 s


In [48]:
%%time
degree_df=pd.DataFrame(directed_degree.items(),columns=['node_id','directed_degree'])
df=df.merge(degree_df,left_on='node1_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_source'},inplace=True)

df=df.merge(degree_df,left_on='node2_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_target'},inplace=True)


CPU times: user 5min 47s, sys: 52.8 s, total: 6min 40s
Wall time: 6min 3s


In [49]:
%%time
temp=pd.DataFrame(directed_degree_in.items(),columns=['node_id','directed_degree'])
df=df.merge(temp,left_on='node1_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_source_in'},inplace=True)


df=df.merge(temp,left_on='node2_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_target_in'},inplace=True)


CPU times: user 10min 33s, sys: 56.5 s, total: 11min 30s
Wall time: 10min 51s


In [50]:
%%time
temp=pd.DataFrame(directed_degree_out.items(),columns=['node_id','directed_degree'])
df=df.merge(temp,left_on='node1_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_source_out'},inplace=True)


df=df.merge(temp,left_on='node2_id',right_on='node_id',how='left')
df.drop('node_id',axis=1,inplace=True)
df.rename(columns={'directed_degree':'directed_degree_target_out'},inplace=True)

CPU times: user 6min 11s, sys: 1min 9s, total: 7min 21s
Wall time: 6min 32s


In [52]:
df.iloc[:,2:].to_pickle('directed_degrees.pkl')

,directed_degree_source,directed_degree_target,directed_degree_source_in,directed_degree_target_in,directed_degree_source_out,directed_degree_target_out
0,0.000005,9.680220e-07,0.000001,9.680220e-07,0.000004,0.000000e+00
1,0.000007,1.452033e-06,0.000002,1.452033e-06,0.000005,0.000000e+00
2,0.000007,3.388077e-06,0.000004,2.057047e-06,0.000003,1.331030e-06
3,0.000009,1.197927e-05,0.000004,5.203118e-06,0.000006,6.776154e-06
4,0.000010,1.573036e-06,0.000003,6.050137e-07,0.000007,9.680220e-07
